In [19]:
from prefixspan import PrefixSpan
from constants import *
from utility import *

In [20]:
instances = []
for split in splits:
    file_path = os.path.join(data_path, f"libriheavy_clean_{split}.jsonl")
    print("reading: ", file_path)
    with open(file_path, "r") as file:
        lines = file.readlines()
        for line in lines:
            jsonObj = json.loads(line.strip())
            jsonObj["match_result"] = read_match_result(jsonObj["match_result"])
            instances.append(jsonObj)
print("total number of instances: ", len(instances))

reading:  /home/mazhang/pt/libriheavy/libriheavy_clean_small.jsonl
reading:  /home/mazhang/pt/libriheavy/libriheavy_clean_medium.jsonl
reading:  /home/mazhang/pt/libriheavy/libriheavy_clean_dev.jsonl
reading:  /home/mazhang/pt/libriheavy/libriheavy_clean_test_clean.jsonl
reading:  /home/mazhang/pt/libriheavy/libriheavy_clean_test_clean_large.jsonl
reading:  /home/mazhang/pt/libriheavy/libriheavy_clean_test_other.jsonl
reading:  /home/mazhang/pt/libriheavy/libriheavy_clean_test_other_large.jsonl
total number of instances:  1277227


In [25]:
mismatch_sequence = []
count_long_mismatch = 0
threshold_long_mismatch = 6
for ins in instances:
    mismatch = ins["mismatched_item"]
    for item in mismatch:
        temp_mismatch = []
        item = item.split("->")
        if len(item[0][1:-1]) != 0:
            temp = item[0][1:-1].split(" ")
            temp_new = []
            for word in temp:
                if word.isnumeric():
                    temp_new.append("<num>")
                else:
                    temp_new.append(word)
            temp_mismatch.extend(temp_new)
        temp_mismatch.append("|")

        if len(item[1][1:-1]) != 0:
            temp = item[1][1:-1].split(" ")
            temp_mismatch.extend(temp)
        mismatch_sequence.append(temp_mismatch)
        temp_mismatch = [x for x in temp_mismatch if x.isalpha() or x=="|"]
        if len(temp_mismatch)>threshold_long_mismatch:
            count_long_mismatch += 1
            if count_long_mismatch<1000:
                print(temp_mismatch)

print(len(mismatch_sequence))
print(count_long_mismatch)

['to', 'the', 'virgins', 'to', 'make', 'much', 'of', 'time', '|', 'erlockstatt', 'org', 'by', 'karen', 'savage']
['outlaw', 's', 'song', 'the', 'chough', '|', 'chaff']
['cent', 'prayer', 'for', 'indifference', '|', 'read', 'filibrevox', 'dot', 'org', 'by', 'karen', 'savage']
['b', 'sheep', 'and', 'lambs', '|', 'read', 'philipprevoxt', 'org', 'by', 'karen', 'savage']
['and', 'the', 'opposite', 'neighbor', 'cried', 'out', 'something', 'about', 'whips', 'and', 'the', 'wind', 'was', 'very', 'high', '|']
['the', 'peterkins', 'at', 'home', '|', 'and', 'chapter', 'four']
['|', 'and', 'of', 'chapter', 'five', 'chapter', 'six']
['|', 'end', 'of', 'chapter', 'seven', 'chapter', 'eight']
['|', 'end', 'of', 'chapter', 'seven', 'chapter', 'eight']
['|', 'end', 'of', 'chapter', 'eleven', 'chapter', 'twelve']
['ann', 'maria', 'bromwick', '|', 'anne', 'marie', 'bronwick']
['|', 'of', 'it', 'she', 'could', 'only', 'think', 'of']
['j', 'ai', '|', 'the', 'jai', 'livre', 'oste']
['livre', 'as', 'tu', 'le'

In [4]:
mismatch_sequence[:10]

[['.', '"', '|'],
 ['an', "'", '|', 'and'],
 ['?', '"', '|'],
 ['.', '|'],
 [',', '|'],
 ['.', '"', '|'],
 [',', '|'],
 [';', '|'],
 ['?', '"', '|'],
 [',', '|']]

In [6]:
sp_written = PrefixSpan(mismatch_sequence)

In [11]:
result = sp_written.topk(5000)

In [12]:
result_new = []
for r in result:
    if "|" in r[1]:
        result_new.append(r)

In [13]:
result_new

[(8456343, ['|']),
 (3749628, [',', '|']),
 (2571205, ['.', '|']),
 (1320569, ['"', '|']),
 (467193, ['.', '"', '|']),
 (351777, [';', '|']),
 (299417, [',', '"', '|']),
 (258986, ["'", '|']),
 (245657, ['?', '|']),
 (219692, ['!', '|']),
 (135780, ['"', '"', '|']),
 (130637, ['?', '"', '|']),
 (104433, ['_', '|']),
 (100654, [':', '|']),
 (85570, ['!', '"', '|']),
 (83101, ['<num>', '|']),
 (71198, ['.', '"', '"', '|']),
 (60682, ['|', 'mister']),
 (60670, ['mr', '|']),
 (60169, ['mr', '|', 'mister']),
 (50568, ['.', '|', 'mister']),
 (50035, ['mr', '.', '|']),
 (49601, ['mr', '.', '|', 'mister']),
 (48786, [',', "'", '|']),
 (48509, ['.', "'", '|']),
 (44103, ['.', '.', '|']),
 (39032, ['?', '"', '"', '|']),
 (33770, [':', '"', '|']),
 (32892, ['the', '|']),
 (31079, ['|', "'"]),
 (30827, ['|', 'the']),
 (30713, ['mrs', '|']),
 (30298, ['|', 'missus']),
 (29931, ['mrs', '|', 'missus']),
 (29463, [')', '|']),
 (29227, ['(', '|']),
 (28519, [',', ',', '|']),
 (28277, ['|', 'and']),
 (2

In [15]:
for r in result_new:
    print("{}: {{{}}}->{{{}}}".format(r[0], " ".join(r[1][:r[1].index("|")]), " ".join(r[1][r[1].index("|")+1:])))

8456343: {}->{}
3749628: {,}->{}
2571205: {.}->{}
1320569: {"}->{}
467193: {. "}->{}
351777: {;}->{}
299417: {, "}->{}
258986: {'}->{}
245657: {?}->{}
219692: {!}->{}
135780: {" "}->{}
130637: {? "}->{}
104433: {_}->{}
100654: {:}->{}
85570: {! "}->{}
83101: {<num>}->{}
71198: {. " "}->{}
60682: {}->{mister}
60670: {mr}->{}
60169: {mr}->{mister}
50568: {.}->{mister}
50035: {mr .}->{}
49601: {mr .}->{mister}
48786: {, '}->{}
48509: {. '}->{}
44103: {. .}->{}
39032: {? " "}->{}
33770: {: "}->{}
32892: {the}->{}
31079: {}->{'}
30827: {}->{the}
30713: {mrs}->{}
30298: {}->{missus}
29931: {mrs}->{missus}
29463: {)}->{}
29227: {(}->{}
28519: {, ,}->{}
28277: {}->{and}
27165: {, .}->{}
26779: {}->{a}
25551: {mrs .}->{}
25375: {.}->{missus}
25125: {a}->{}
24942: {mrs .}->{missus}
23831: {<num> <num>}->{}
23213: {. ,}->{}
20638: {s}->{}
20008: {. . .}->{}
19507: {and}->{}
19358: {}->{s}
19155: {' '}->{}
19145: {; "}->{}
18981: {<num> .}->{}
18762: {. <num>}->{}
17984: {' s}->{}
16475: {<num> :}

In [18]:
for ins in instances:
    mismatch = ins["mismatched_item"]
    for item in mismatch:
        item = item.split("->")
        if len(item[0][1:-1]) != 0:
            temp = item[0][1:-1].split(" ")
            if "page" in temp:
                print("written: ", ins["written"])
                print("spoken: ", ins["spoken"])
                print("mismatches: ", mismatch)

written:  And Agamemnon made a copy of the programme for each member of the family. THE PETERKINS ARE MOVED.  Page 126. The first thing to be done was to buy the parlor carpets. Elizabeth Eliza had already looked at some in Boston, and the next morning she went, by an early train, with her father, Agamemnon, and Solomon John, to decide upon them.
spoken:  AND AGAMEMNON MADE A COPY OF THE PROGRAM FOR EACH MEMBER OF THE FAMILY THE FIRST THING TO BE DONE WAS TO BUY PARLOR CARPETS ELIZABETH ELIZA HAD ALREADY LOOKED AT SOME IN BOSTON AND THE NEXT MORNING SHE WENT BY AN EARLY TRAIN WITH HER FATHER AGAMEMNON AND SOLOMON JOHN TO DECIDE UPON THEM
mismatches:  ['{programme}->{program}', '{. the peterkins are moved . page 126 .}->{}', '{the}->{}', '{.}->{}', '{,}->{}', '{,}->{}', '{,}->{}', '{,}->{}', '{,}->{}', '{,}->{}', '{.}->{}']
written:  page%>\ SECTION VI THE STABLE SOLUTIONS OF THIS INSTABILITY : The three stable social arrangements which alone can take the place of unstable Capitalism Th